In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import time
import csv
import matplotlib.pyplot as plt

# إعداد Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# فتح الكاميرا
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise Exception("❌ لم يتمكن من فتح الكاميرا!")

# دالة لتصنيف اللكمات
def classify_punch(wrist, elbow, prev_wrist_pos, prev_angle, fps):
    dx = wrist[0] - elbow[0]
    dy = wrist[1] - elbow[1]
    angle = np.degrees(np.arctan2(dy, dx))

    # حساب السرعة
    speed = 0
    moved_forward = False
    if prev_wrist_pos is not None:
        prev_wx, prev_wy = prev_wrist_pos
        distance = np.sqrt((wrist[0] - prev_wx)**2 + (wrist[1] - prev_wy)**2)
        time_per_frame = 1 / fps
        speed = distance / time_per_frame
        moved_forward = (prev_wx - wrist[0]) > 0.005

    angle_change = 0
    if prev_angle is not None:
        angle_change = abs(angle - prev_angle)

    punch_type = "non"
    if moved_forward and angle_change > 5:
        if -30 <= angle <= 30:
            punch_type = "jab"
        elif 80 <= angle <= 110 or -110 <= angle <= -80:
            punch_type = "hook"
        elif angle > 120 or angle < -120:
            punch_type = "upper"

    strength = "none"
    if punch_type != "non":
        if speed > 0.5:
            strength = "strong"
        elif speed > 0.2:
            strength = "medium"
        else:
            strength = "weak"

    return punch_type, angle, speed, strength

# متغيرات للتخزين
prev_wrist_pos = None
prev_angle = None
fps = cap.get(cv2.CAP_PROP_FPS) or 30
session_data = []
punch_counts = {"jab": 0, "hook": 0, "upper": 0}
strength_counts = {"strong": 0, "medium": 0, "weak": 0}
reaction_times = []
last_punch_time = None
correct_punches = 0
total_punches = 0

# Mediapipe
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        label = "non"
        angle = 0
        speed = 0
        strength = "none"

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            right_wrist = (landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y)
            right_elbow = (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y)

            label, angle, speed, strength = classify_punch(
                right_wrist, right_elbow, prev_wrist_pos, prev_angle, fps)

            if label != "non":
                correct_punches += 1
                punch_counts[label] += 1
                strength_counts[strength] += 1

                if last_punch_time is not None:
                    reaction_time = time.time() - last_punch_time
                    reaction_times.append(reaction_time)
                last_punch_time = time.time()

                session_data.append([label, angle, speed, strength])

            total_punches += 1
            prev_wrist_pos = right_wrist
            prev_angle = angle
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # عرض البيانات على الشاشة
        cv2.putText(frame, f"Punch: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Angle: {angle:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.putText(frame, f"Speed: {speed:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        cv2.putText(frame, f"Strength: {strength}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        cv2.imshow("Live Punch Classification", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# حسابات الأداء
accuracy = correct_punches / total_punches * 100 if total_punches else 0
avg_reaction_time = np.mean(reaction_times) if reaction_times else 0
avg_power = (strength_counts["strong"] * 1 + strength_counts["medium"] * 0.5) / \
            sum(strength_counts.values()) * 100 if sum(strength_counts.values()) else 0

# حفظ البيانات CSV
with open("/mnt/data/punch_summary.csv", mode="w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Punch", "Angle", "Speed", "Strength"])
    writer.writerows(session_data)

# رسم الأداء
plt.figure(figsize=(10, 5))
plt.bar(punch_counts.keys(), punch_counts.values(), color='orange')
plt.title("Punch Type Distribution")
plt.xlabel("Punch Type")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.savefig("/mnt/data/punch_distribution.png")

(punch_counts, strength_counts, accuracy, avg_power, avg_reaction_time)

KeyboardInterrupt: 